In [2]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [3]:
data = pd.read_csv(path+'data/merged.csv')

In [14]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season != 2021]
X_train = np.asarray(train.drop(['driver', 'podium', 'circuit_id','country','date','date_of_birth','nationality','constructor', 'q1', 'q2', 'q3', 'lat', 'long', 'season', 'round', 'pos'], axis = 1).values)
y_train = np.asarray(train.podium.values)

In [16]:
svm_wins= svm.SVC(kernel="linear",gamma='auto') # Initialize the SVM model
svm_wins.fit(X_train, y_train) # Train the SVM model

SVC(gamma='auto', kernel='linear')

In [53]:
race = np.asarray(df.query('season == 2021 & round == 13').drop(['driver', 'podium', 'circuit_id','country','date','date_of_birth','nationality','constructor', 'q1', 'q2', 'q3', 'lat', 'long', 'season', 'round', 'pos'], axis = 1).values)
labels = np.asarray(df.query('season == 2021 & round == 1').podium.values)

print(svm_wins.predict(race))
print(labels)

[1 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
